In [1]:
import numpy as np
import pandas as pd
import os
import re
import string
import random

In [3]:
# opening the train_source file in read mode
my_file = open("train-source.txt", "r", encoding='UTF-8')

# reading the file
data = my_file.read()

train_source_list = data.replace('\n'," ").split('<s>')

my_file.close()

#Remove End Sentence Tags
for item in range(len(train_source_list)):
  train_source_list[item] = train_source_list[item].replace('</s>', '')


In [ ]:
train_source_list

In [4]:
# opening the train-target file in read mode
my_file = open("train-target.txt", "r", encoding='UTF-8')

# reading the file
data = my_file.read()

train_target_list = data.replace('\n'," ").split('<s>')

my_file.close()

#Remove End Sentence Tags
for item in range(len(train_target_list)):
  train_target_list[item] = train_target_list[item].replace('</s>', '')

In [6]:
# opening the test-target file in read mode
my_file = open("test-source.txt", "r", encoding='UTF-8')

# reading the file
data = my_file.read()

test_source_list = data.replace('\n'," ").split('<s>')

my_file.close()

#Remove End Sentence Tags
for item in range(len(test_source_list)):
  test_source_list[item] = test_source_list[item].replace('</s>', '')

# opening the test-target file in read mode
my_file = open("test-target.txt", "r", encoding='UTF-8')

# reading the file
data = my_file.read()

test_target_list = data.replace('\n'," ").split('<s>')

my_file.close()

#Remove End Sentence Tags
for item in range(len(test_target_list)):
  test_target_list[item] = test_target_list[item].replace('</s>', '')

In [7]:
print("length of training sets Source: ", len(train_source_list), " Target :", len(train_target_list))
print("length of test sets Source: ", len(test_source_list), " Target :", len(test_target_list))

length of training sets Source:  45172  Target : 45172
length of test sets Source:  1001  Target : 1001


In [10]:
#Anything goes ref: https://machinelearningmastery.com/calculate-bleu-score-for-text-python/
from nltk.translate.bleu_score import sentence_bleu,corpus_bleu


In [11]:
#For Train
reference_train = train_source_list
candidate_train = train_target_list[1]

print('Cumulative 1-gram: %f' % sentence_bleu(reference_train, candidate_train, weights=(1, 0, 0, 0)))
print('Cumulative 2-gram: %f' % sentence_bleu(reference_train, candidate_train, weights=(0.5, 0.5, 0, 0)))
print('Cumulative 3-gram: %f' % sentence_bleu(reference_train, candidate_train, weights=(0.33, 0.33, 0.33, 0)))
print('Cumulative 4-gram: %f' % sentence_bleu(reference_train, candidate_train, weights=(0.25, 0.25, 0.25, 0.25)))

Cumulative 1-gram: 1.000000
Cumulative 2-gram: 1.000000
Cumulative 3-gram: 1.000000
Cumulative 4-gram: 1.000000


In [12]:
#For Test
reference_test = test_source_list
candidate_test = test_target_list[1]

print('Cumulative 1-gram: %f' % sentence_bleu(reference_test, candidate_test, weights=(1, 0, 0, 0)))
print('Cumulative 2-gram: %f' % sentence_bleu(reference_test, candidate_test, weights=(0.5, 0.5, 0, 0)))
print('Cumulative 3-gram: %f' % sentence_bleu(reference_test, candidate_test, weights=(0.33, 0.33, 0.33, 0)))
print('Cumulative 4-gram: %f' % sentence_bleu(reference_test, candidate_test, weights=(0.25, 0.25, 0.25, 0.25)))

Cumulative 1-gram: 1.000000
Cumulative 2-gram: 1.000000
Cumulative 3-gram: 0.989740
Cumulative 4-gram: 0.963897


In [13]:
#From scratch: https://stackoverflow.com/questions/56968434/bleu-score-in-python-from-scratch
import numpy as np
from collections import Counter
import math

In [14]:
def n_gram_generator(sentence,n= 2,n_gram= False):
    '''
    N-Gram generator with parameters sentence
    n is for number of n_grams
    The n_gram parameter removes repeating n_grams
    '''
    sentence = sentence.lower()  # converting to lower case
    sent_arr = np.array(sentence.split())  # split to string arrays
    length = len(sent_arr)

    word_list = []
    for i in range(length+1):
        if i < n:
            continue
        word_range = list(range(i-n,i))
        s_list = sent_arr[word_range]
        string = ' '.join(s_list)  # converting list to strings
        word_list.append(string) # append to word_list
        if n_gram:
            word_list = list(set(word_list))
    return word_list

In [19]:
def bleu_score(original, reference):
    '''
    Bleu score function given a orginal and a machine translated sentences
    '''
    rf_length = len(reference.split())
    o_length  = len(original.split())

    # Brevity Penalty
    if rf_length > o_length:
        BP=1
    else:
        penality=1-(rf_length/o_length)
        BP = np.exp(penality)

    # Clipped precision
    clipped_precision_score = []
    for ngram_level in range(1, 4):  # 1-gram to 4-gram
        
        
        original_ngram_list = n_gram_generator(original, ngram_level)
        original_n_gram = Counter(original_ngram_list)
        
        reference_ngram_list = n_gram_generator(reference, ngram_level)
        reference_n_gram = Counter(reference_ngram_list)
        
        
        num_ngrams_in_translation = sum(reference_n_gram.values())  # number of ngrams in translation
        
        # iterate the unique ngrams in translation (candidate)
        for j in reference_n_gram:
            
            if j in original_n_gram:  # if found in reference
                
                if reference_n_gram[j] > original_n_gram[j]:  # CLIPPING - if found in translation more than in source, clip
                    reference_n_gram[j] = original_n_gram[j]
                    
            else:
                reference_n_gram[j] = 0

        #print (sum(machine_n_gram.values()), c)
        clipped_precision_score.append(float(sum(reference_n_gram.values())) / num_ngrams_in_translation)

    #print (clipped_precision_score)

    weights = [0.25]*4

    s = (w_i * math.log(p_i) for w_i, p_i in zip(weights, clipped_precision_score))
    s = BP * math.exp(math.fsum(s))
    return s

In [26]:
#For Train Files
original_train = train_source_list[1]
reference_train = train_target_list[1]

print ("BLEU Score from scratch for Train files : ", bleu_score(original_train, reference_train))

BLEU Score from scratch for Train files :  0.36787632499277756


In [27]:
#For Train Files
original_test = test_source_list[1]
reference_test = test_target_list[1]

print ("BLEU Score from scratch for Test files : ", bleu_score(original_test, reference_test))

BLEU Score from scratch for Test files :  0.47287080450158786
